/opt/conda/envs/ocr_env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


error: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1721302265921/work/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


ValueError: could not broadcast input array from shape (5,5) into shape (5,5,3)

In [3]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from skimage import exposure, restoration
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Load the pre-trained Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

# Preprocessing Functions
def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 30, 30, 7, 21)

def deblur_image(image):
    psf = np.ones((5, 5)) / 25
    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    deconvolved, _ = restoration.unsupervised_wiener(image, psf)
    return (deconvolved * 255).astype(np.uint8)

def enhance_contrast(image):
    enhanced_image = exposure.equalize_adapthist(image, clip_limit=0.03)
    return (enhanced_image * 255).astype(np.uint8)

def remove_stains(image):
    mask = cv2.inRange(image, np.array([0, 0, 0]), np.array([60, 60, 60]))
    inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
    return inpainted_image

def preprocess_image(image):
    image = np.array(image)
    
    denoised = denoise_image(image)
    deblurred = deblur_image(denoised)
    
    contrast_enhanced = enhance_contrast(deblurred)
    if len(contrast_enhanced.shape) == 2:
        contrast_enhanced = cv2.cvtColor(contrast_enhanced, cv2.COLOR_GRAY2BGR)
        
    restored_image = remove_stains(contrast_enhanced)
    if len(restored_image.shape) == 2:
        restored_image = cv2.cvtColor(restored_image, cv2.COLOR_GRAY2BGR)
    
    restored_image = Image.fromarray(restored_image)
    restored_image = ImageEnhance.Contrast(restored_image).enhance(2)
    restored_image = ImageEnhance.Brightness(restored_image).enhance(1.5)
    
    return restored_image

def recognize_text(image):
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

def find_best_orientation(image_path):
    img = Image.open(image_path).convert("RGB")
    preprocessed_image = preprocess_image(img)

    angles = [0, 90, 180, 270]
    best_score = 0
    best_transformation = None

    for angle in angles:
        rotated_image = preprocessed_image.rotate(angle, expand=True)
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Angle: {angle}, Readability Score: {readability_score}")

        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image

    flipped_image = preprocessed_image.transpose(Image.FLIP_LEFT_RIGHT)
    for angle in angles:
        rotated_image = flipped_image.rotate(angle, expand=True)
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Flipped Angle: {angle}, Readability Score: {readability_score}")

        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image

    return best_transformation

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
best_image = find_best_orientation(image_path)
best_image.save('dj/best_oriented_image.jpg')
print("Best oriented image saved to 'dj/best_oriented_image.jpg'")


/opt/conda/envs/ocr_env2/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Angle: 0, Readability Score: 0
Angle: 90, Readability Score: 0
Angle: 180, Readability Score: 0
Angle: 270, Readability Score: 0
Flipped Angle: 0, Readability Score: 0
Flipped Angle: 90, Readability Score: 0
Flipped Angle: 180, Readability Score: 0
Flipped Angle: 270, Readability Score: 0


AttributeError: 'NoneType' object has no attribute 'save'

In [4]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from skimage import exposure, restoration
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Load the pre-trained Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

# Preprocessing Functions
def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 30, 30, 7, 21)

def deblur_image(image):
    psf = np.ones((5, 5)) / 25
    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    deconvolved, _ = restoration.unsupervised_wiener(image, psf)
    return (deconvolved * 255).astype(np.uint8)

def enhance_contrast(image):
    enhanced_image = exposure.equalize_adapthist(image, clip_limit=0.03)
    return (enhanced_image * 255).astype(np.uint8)

def remove_stains(image):
    mask = cv2.inRange(image, np.array([0, 0, 0]), np.array([60, 60, 60]))
    inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
    return inpainted_image

def preprocess_image(image):
    image = np.array(image)
    
    denoised = denoise_image(image)
    deblurred = deblur_image(denoised)
    
    contrast_enhanced = enhance_contrast(deblurred)
    if len(contrast_enhanced.shape) == 2:
        contrast_enhanced = cv2.cvtColor(contrast_enhanced, cv2.COLOR_GRAY2BGR)
        
    restored_image = remove_stains(contrast_enhanced)
    if len(restored_image.shape) == 2:
        restored_image = cv2.cvtColor(restored_image, cv2.COLOR_GRAY2BGR)
    
    restored_image = Image.fromarray(restored_image)
    restored_image = ImageEnhance.Contrast(restored_image).enhance(2)
    restored_image = ImageEnhance.Brightness(restored_image).enhance(1.5)
    
    return restored_image

def recognize_text(image):
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values, max_new_tokens=50)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

def find_best_orientation(image_path):
    img = Image.open(image_path).convert("RGB")
    preprocessed_image = preprocess_image(img)

    angles = [0, 90, 180, 270]
    best_score = 0
    best_transformation = None

    for angle in angles:
        rotated_image = preprocessed_image.rotate(angle, expand=True)
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Angle: {angle}, Readability Score: {readability_score}")

        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image

    flipped_image = preprocessed_image.transpose(Image.FLIP_LEFT_RIGHT)
    for angle in angles:
        rotated_image = flipped_image.rotate(angle, expand=True)
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Flipped Angle: {angle}, Readability Score: {readability_score}")

        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image

    if best_transformation is None:
        best_transformation = preprocessed_image

    return best_transformation

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
best_image = find_best_orientation(image_path)
if best_image is not None:
    best_image.save('dj/best_oriented_image.jpg')
    print("Best oriented image saved to 'dj/best_oriented_image.jpg'")
else:
    print("No suitable transformation found.")


Angle: 0, Readability Score: 0
Angle: 90, Readability Score: 0
Angle: 180, Readability Score: 0
Angle: 270, Readability Score: 0
Flipped Angle: 0, Readability Score: 0
Flipped Angle: 90, Readability Score: 0
Flipped Angle: 180, Readability Score: 0
Flipped Angle: 270, Readability Score: 0
Best oriented image saved to 'dj/best_oriented_image.jpg'


In [7]:
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Load the pre-trained Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

# Preprocessing function
def preprocess_image(image):
    # Resize image to improve OCR accuracy
    resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    
    # Convert to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    
    # Additional noise removal using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    denoised = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    # Apply CLAHE to improve contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(denoised)
    
    # Convert back to RGB
    enhanced_image_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)
    
    return enhanced_image_rgb

# Function to recognize text using Donut model
def recognize_text(image):
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values, max_new_tokens=50)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

# Function to find the best orientation
def find_best_orientation(image_path):
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    
    preprocessed_image = preprocess_image(img)
    angles = [0, 90, 180, 270]
    best_score = 0
    best_transformation = None
    
    for angle in angles:
        rotated_image = Image.fromarray(np.rot90(preprocessed_image, k=angle // 90))
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Angle: {angle}, Readability Score: {readability_score}")
        
        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image
    
    flipped_image = np.fliplr(preprocessed_image)
    for angle in angles:
        rotated_image = Image.fromarray(np.rot90(flipped_image, k=angle // 90))
        recognized_text = recognize_text(rotated_image)
        readability_score = len(recognized_text)
        print(f"Flipped Angle: {angle}, Readability Score: {readability_score}")
        
        if readability_score > best_score:
            best_score = readability_score
            best_transformation = rotated_image
    
    if best_transformation is None:
        best_transformation = Image.fromarray(preprocessed_image)
    
    return best_transformation

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
best_image = find_best_orientation(image_path)
if best_image is not None:
    best_image.save('dj/best_oriented_image.jpg')
    print("Best oriented image saved to 'dj/best_oriented_image.jpg'")
else:
    print("No suitable transformation found.")


Angle: 0, Readability Score: 0
Angle: 90, Readability Score: 0
Angle: 180, Readability Score: 0
Angle: 270, Readability Score: 0
Flipped Angle: 0, Readability Score: 0
Flipped Angle: 90, Readability Score: 0
Flipped Angle: 180, Readability Score: 0
Flipped Angle: 270, Readability Score: 0
Best oriented image saved to 'dj/best_oriented_image.jpg'


In [11]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance

def preprocess_image(image_path):
    try:
        print("Loading Image...")
        image = Image.open(image_path).convert("RGB")
        image_np = np.array(image)
        
        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image_np, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        print("Resized Image Shape:", resized_image.shape)

        # Convert to grayscale
        gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        print("Converted to Grayscale")

        # Apply GaussianBlur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        print("Applied GaussianBlur")

        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
        print("Applied Adaptive Thresholding")

        # Additional noise removal using morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        denoised = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        print("Applied Morphological Operations")

        # Apply CLAHE to improve contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(denoised)
        print("Applied CLAHE")

        # Save preprocessed image
        preprocessed_image = Image.fromarray(enhanced_image)
        preprocessed_image.save('dj/preprocessed_image.jpg')
        print("Preprocessed image saved to 'dj/preprocessed_image.jpg'")

    except Exception as e:
        print("Error during preprocessing:", e)

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
preprocess_image(image_path)

Loading Image...
Resized Image Shape: (1182, 886, 3)
Converted to Grayscale
Applied GaussianBlur
Applied Adaptive Thresholding
Applied Morphological Operations
Applied CLAHE
Preprocessed image saved to 'dj/preprocessed_image.jpg'


In [12]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageOps

def preprocess_image(image_path, output_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")

        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        
        # Convert to grayscale
        gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        
        # Apply GaussianBlur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
        
        # Additional noise removal using morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        denoised = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        
        # Apply CLAHE to improve contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(denoised)
        
        # Convert back to PIL Image for further enhancements
        pil_image = Image.fromarray(enhanced_image)
        
        # Enhance contrast and brightness
        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)
        
        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)
        
        # Save the preprocessed image
        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")
        
    except Exception as e:
        print("Error during preprocessing:", e)

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)


Preprocessed image saved to dj/preprocessed_image.jpg


In [15]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

# Define the sharpening kernel
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Enhance edges using Canny edge detection
def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Load and use a pre-trained deep learning model for image enhancement
def enhance_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Identity()
    model = model.to(device)
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        enhanced_tensor = model(img_tensor)
    enhanced_image = transforms.ToPILImage()(enhanced_tensor.squeeze().cpu())
    return enhanced_image

def preprocess_image(image_path, output_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")

        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        
        # Sharpen the image
        sharpened_image = sharpen_image(resized_image)
        
        # Enhance edges
        edges = edge_detection(sharpened_image)
        
        # Apply deep learning enhancement
        enhanced_image = enhance_image(image_path)

        # Convert the enhanced image back to OpenCV format
        enhanced_image = cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)
        
        # Combine the processed images
        combined_image = cv2.addWeighted(sharpened_image, 0.5, edges, 0.5, 0)
        combined_image = cv2.addWeighted(combined_image, 0.7, enhanced_image, 0.3, 0)

        # Convert back to PIL Image for further enhancements
        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
        
        # Enhance contrast and brightness
        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)
        
        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)
        
        # Save the preprocessed image
        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")
        
    except Exception as e:
        print("Error during preprocessing:", e)

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)


Error during preprocessing: pic should be 2/3 dimensional. Got 1 dimensions.


In [17]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

# Define the sharpening kernel
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Enhance edges using Canny edge detection
def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Load and use a pre-trained deep learning model for image enhancement
def enhance_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Identity()
    model = model.to(device)
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        enhanced_tensor = model(img_tensor)
    
    enhanced_image = transforms.ToPILImage()(enhanced_tensor.squeeze().cpu())
    return enhanced_image

def preprocess_image(image_path, output_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")

        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        
        # Sharpen the image
        sharpened_image = sharpen_image(resized_image)
        
        # Enhance edges
        edges = edge_detection(sharpened_image)
        
        # Apply deep learning enhancement
        enhanced_image = enhance_image(image_path)

        # Convert the enhanced image back to OpenCV format
        enhanced_image = cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)
        
        # Combine the processed images
        combined_image = cv2.addWeighted(sharpened_image, 0.5, edges, 0.5, 0)
        combined_image = cv2.addWeighted(combined_image, 0.7, enhanced_image, 0.3, 0)

        # Convert back to PIL Image for further enhancements
        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
        
        # Enhance contrast and brightness
        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)
        
        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)
        
        # Save the preprocessed image
        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")
        
    except Exception as e:
        print("Error during preprocessing:", e)


# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)


Error during preprocessing: pic should be 2/3 dimensional. Got 1 dimensions.


In [18]:

import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

# Define the sharpening kernel
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Enhance edges using Canny edge detection
def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Load and use a pre-trained deep learning model for image enhancement
def enhance_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Identity()
    model = model.to(device)
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        enhanced_tensor = model(img_tensor)
    
    enhanced_image = transforms.ToPILImage()(enhanced_tensor.squeeze().cpu())
    return enhanced_image

def preprocess_image(image_path, output_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")
        print(f"Original image shape: {image.shape}")

        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        print(f"Resized image shape: {resized_image.shape}")
        
        # Sharpen the image
        sharpened_image = sharpen_image(resized_image)
        print(f"Sharpened image shape: {sharpened_image.shape}")
        
        # Enhance edges
        edges = edge_detection(sharpened_image)
        print(f"Edges image shape: {edges.shape}")
        
        # Apply deep learning enhancement
        enhanced_image = enhance_image(image_path)
        print(f"Enhanced image type: {type(enhanced_image)}")
        
        # Convert the enhanced image back to OpenCV format
        enhanced_image = cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)
        print(f"Enhanced image shape: {enhanced_image.shape}")
        
        # Combine the processed images
        combined_image = cv2.addWeighted(sharpened_image, 0.5, edges, 0.5, 0)
        print(f"Combined image shape after adding edges: {combined_image.shape}")
        combined_image = cv2.addWeighted(combined_image, 0.7, enhanced_image, 0.3, 0)
        print(f"Combined image shape after adding enhanced image: {combined_image.shape}")

        # Convert back to PIL Image for further enhancements
        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
        print(f"PIL image type: {type(pil_image)}")
        
        # Enhance contrast and brightness
        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)
        
        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)
        
        # Save the preprocessed image
        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")
        
    except Exception as e:
        print("Error during preprocessing:", e)




image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)

Original image shape: (591, 443, 3)
Resized image shape: (1182, 886, 3)
Sharpened image shape: (1182, 886, 3)
Edges image shape: (1182, 886)
Error during preprocessing: pic should be 2/3 dimensional. Got 1 dimensions.


In [19]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

# Define the sharpening kernel
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Enhance edges using Canny edge detection
def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Load and use a pre-trained deep learning model for image enhancement
def enhance_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Identity()
    model = model.to(device)
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        enhanced_tensor = model(img_tensor)
    
    enhanced_image = transforms.ToPILImage()(enhanced_tensor.squeeze().cpu())
    return enhanced_image

def preprocess_image(image_path, output_path):
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")
        print(f"Original image shape: {image.shape}")

        # Resize image to improve OCR accuracy
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        print(f"Resized image shape: {resized_image.shape}")
        
        # Sharpen the image
        sharpened_image = sharpen_image(resized_image)
        print(f"Sharpened image shape: {sharpened_image.shape}")
        
        # Enhance edges
        edges = edge_detection(sharpened_image)
        print(f"Edges image shape: {edges.shape}")
        
        # Convert edges to three channels
        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        print(f"Edges colored image shape: {edges_colored.shape}")
        
        # Apply deep learning enhancement
        enhanced_image = enhance_image(image_path)
        print(f"Enhanced image type: {type(enhanced_image)}")
        
        # Convert the enhanced image back to OpenCV format
        enhanced_image = cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)
        print(f"Enhanced image shape: {enhanced_image.shape}")
        
        # Combine the processed images
        combined_image = cv2.addWeighted(sharpened_image, 0.5, edges_colored, 0.5, 0)
        print(f"Combined image shape after adding edges: {combined_image.shape}")
        combined_image = cv2.addWeighted(combined_image, 0.7, enhanced_image, 0.3, 0)
        print(f"Combined image shape after adding enhanced image: {combined_image.shape}")

        # Convert back to PIL Image for further enhancements
        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
        print(f"PIL image type: {type(pil_image)}")
        
        # Enhance contrast and brightness
        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)
        
        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)
        
        # Save the preprocessed image
        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")
        
    except Exception as e:
        print("Error during preprocessing:", e)



image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)

Original image shape: (591, 443, 3)
Resized image shape: (1182, 886, 3)
Sharpened image shape: (1182, 886, 3)
Edges image shape: (1182, 886)
Edges colored image shape: (1182, 886, 3)
Error during preprocessing: pic should be 2/3 dimensional. Got 1 dimensions.


In [23]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

def enhance_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Identity()
    model = model.to(device)
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        enhanced_tensor = model(img_tensor)
    
    enhanced_image = transforms.ToPILImage()(enhanced_tensor.squeeze().cpu())
    return enhanced_image

def preprocess_image(image_path, output_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not open or find the image.")
        print(f"Original image shape: {image.shape}")

        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        print(f"Resized image shape: {resized_image.shape}")

        sharpened_image = sharpen_image(resized_image)
        print(f"Sharpened image shape: {sharpened_image.shape}")

        edges = edge_detection(sharpened_image)
        print(f"Edges image shape: {edges.shape}")

        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        print(f"Edges colored image shape: {edges_colored.shape}")

        enhanced_image = enhance_image(image_path)
        print(f"Enhanced image type: {type(enhanced_image)}")

        enhanced_image = enhanced_image.resize(resized_image.shape[1::-1])
        enhanced_image = cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)
        print(f"Enhanced image shape: {enhanced_image.shape}")

        combined_image = cv2.addWeighted(sharpened_image, 0.5, edges_colored, 0.5, 0)
        print(f"Combined image shape after adding edges: {combined_image.shape}")
        combined_image = cv2.addWeighted(combined_image, 0.7, enhanced_image, 0.3, 0)
        print(f"Combined image shape after adding enhanced image: {combined_image.shape}")

        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
        print(f"PIL image type: {type(pil_image)}")

        enhancer = ImageEnhance.Contrast(pil_image)
        pil_image = enhancer.enhance(2)

        enhancer = ImageEnhance.Brightness(pil_image)
        pil_image = enhancer.enhance(1.5)

        pil_image.save(output_path)
        print(f"Preprocessed image saved to {output_path}")

    except Exception as e:
        print("Error during preprocessing:", e)



image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
output_path = 'dj/preprocessed_image.jpg'
preprocess_image(image_path, output_path)

Original image shape: (591, 443, 3)
Resized image shape: (1182, 886, 3)
Sharpened image shape: (1182, 886, 3)
Edges image shape: (1182, 886)
Edges colored image shape: (1182, 886, 3)
Error during preprocessing: pic should be 2/3 dimensional. Got 1 dimensions.


In [24]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance

def preprocess_image(image_path):
    try:
        # Load image
        image = cv2.imread(image_path)
        print("Original image shape:", image.shape)

        # Resize image
        resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        print("Resized image shape:", resized_image.shape)

        # Sharpen the image
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_image = cv2.filter2D(resized_image, -1, kernel)
        print("Sharpened image shape:", sharpened_image.shape)

        # Edge detection
        edges = cv2.Canny(sharpened_image, 100, 200)
        print("Edges image shape:", edges.shape)

        # Convert edges to 3 channels
        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        print("Edges colored image shape:", edges_colored.shape)

        # Combine the sharpened image and edges
        combined_image = cv2.addWeighted(sharpened_image, 0.8, edges_colored, 0.2, 0)

        # Convert to PIL Image
        pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))

        # Enhance the image contrast and brightness
        pil_image = ImageEnhance.Contrast(pil_image).enhance(2)
        pil_image = ImageEnhance.Brightness(pil_image).enhance(1.5)

        # Save the preprocessed image
        preprocessed_path = 'dj/preprocessed_image.jpg'
        pil_image.save(preprocessed_path)
        print("Preprocessed image saved to 'dj/preprocessed_image.jpg'")

        return preprocessed_path

    except Exception as e:
        print("Error during preprocessing:", e)
        return None

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
preprocessed_image_path = preprocess_image(image_path)

Original image shape: (591, 443, 3)
Resized image shape: (1182, 886, 3)
Sharpened image shape: (1182, 886, 3)
Edges image shape: (1182, 886)
Edges colored image shape: (1182, 886, 3)
Preprocessed image saved to 'dj/preprocessed_image.jpg'


In [29]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor, ToPILImage

# Define the Super-Resolution model
class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor):
        super(SuperResolutionNet, self).__init__()
        self.upscale_factor = upscale_factor
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 3 * (upscale_factor ** 2), kernel_size=3, stride=1, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

def enhance_image(image_path, model_path):
    # Load image
    image = Image.open(image_path).convert("RGB")
    print("Original image shape:", image.size)

    # Load model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SuperResolutionNet(upscale_factor=3).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Preprocess image
    image_tensor = ToTensor()(image).unsqueeze(0).to(device)

    with torch.no_grad():
        # Enhance image using the model
        enhanced_tensor = model(image_tensor)
    
    enhanced_image = ToPILImage()(enhanced_tensor.squeeze(0).cpu())
    print("Enhanced image shape:", enhanced_image.size)

    # Convert enhanced image to numpy array for further processing
    enhanced_image_np = np.array(enhanced_image)

    # Apply sharpening
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened_image = cv2.filter2D(enhanced_image_np, -1, kernel)
    print("Sharpened image shape:", sharpened_image.shape)

    # Edge detection
    edges = cv2.Canny(sharpened_image, 100, 200)
    print("Edges image shape:", edges.shape)

    # Convert edges to 3 channels
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    print("Edges colored image shape:", edges_colored.shape)

    # Combine the sharpened image and edges
    combined_image = cv2.addWeighted(sharpened_image, 0.8, edges_colored, 0.2, 0)

    # Convert to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))

    # Enhance the image contrast and brightness
    pil_image = ImageEnhance.Contrast(pil_image).enhance(2)
    pil_image = ImageEnhance.Brightness(pil_image).enhance(1.5)

    return pil_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = 'super_res_model.pth'  # Ensure you have the model file here
enhanced_image = enhance_image(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")



Original image shape: (443, 591)


/tmp/ipykernel_179068/1396399940.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


FileNotFoundError: [Errno 2] No such file or directory: 'super_res_model.pth'

In [3]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor, ToPILImage
import functools

class RRDBNet(nn.Module):
    def __init__(self, in_nc, out_nc, nf, nb, gc=32):
        super(RRDBNet, self).__init__()
        RRDB_block_f = functools.partial(RRDB, nf=nf, gc=gc)
        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)
        self.RRDB_trunk = self.make_layer(RRDB_block_f, nb)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)

    def make_layer(self, block, n_layers):
        layers = []
        for _ in range(n_layers):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk
        fea = self.upconv1(nn.functional.interpolate(fea, scale_factor=2, mode='nearest'))
        fea = self.upconv2(nn.functional.interpolate(fea, scale_factor=2, mode='nearest'))
        out = self.conv_last(self.HRconv(fea))
        return out

class RRDB(nn.Module):
    def __init__(self, nf, gc=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nf, gc)
        self.RDB2 = ResidualDenseBlock_5C(nf, gc)
        self.RDB3 = ResidualDenseBlock_5C(nf, gc)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out * 0.2 + x

class ResidualDenseBlock_5C(nn.Module):
    def __init__(self, nf=64, gc=32):
        super(ResidualDenseBlock_5C, self).__init__()
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=True)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=True)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=True)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=True)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=True)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x5 * 0.2 + x

def enhance_image(image_path, model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RRDBNet(3, 3, 64, 23, gc=32).to(device)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()

    image = Image.open(image_path).convert('RGB')
    transform = ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        sr_tensor = model(image_tensor)

    sr_image = ToPILImage()(sr_tensor.squeeze().cpu())
    return sr_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = '/dj/super_res_model.pth'  # Ensure you have the model file here
enhanced_image = enhance_image(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


/tmp/ipykernel_181103/1179060273.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path), strict=True)


UnpicklingError: invalid load key, '<'.

In [4]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO

# Load pre-trained DIP model
model_url = 'https://github.com/DmitryUlyanov/deep-image-prior/releases/download/1.0/DIP_model.pth'
model_path = '/dj/DIP_model.pth'
response = requests.get(model_url)
with open(model_path, 'wb') as f:
    f.write(response.content)

class DIPModel(torch.nn.Module):
    def __init__(self):
        super(DIPModel, self).__init__()
        # Define model architecture here

    def forward(self, x):
        # Define forward pass here
        return x

def enhance_image_with_dip(image_path, model_path):
    # Load image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    image_tensor = transform(image).unsqueeze(0)

    # Load model
    model = DIPModel()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Enhance image
    with torch.no_grad():
        enhanced_image_tensor = model(image_tensor)

    # Convert tensor back to image
    enhanced_image = transforms.ToPILImage()(enhanced_image_tensor.squeeze(0))
    return enhanced_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
enhanced_image = enhance_image_with_dip(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


/tmp/ipykernel_181103/3366680254.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


UnpicklingError: could not find MARK

In [5]:
import cv2
import numpy as np
from PIL import Image

def enhance_image_with_fsrcnn(image_path, model_path):
    # Load the pre-trained FSRCNN model
    net = cv2.dnn.readNetFromTensorflow(model_path)
    
    # Read and preprocess the image
    image = cv2.imread(image_path)
    h, w, c = image.shape
    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(w, h), mean=(0, 0, 0), swapRB=False, crop=False)
    
    # Perform the super-resolution
    net.setInput(blob)
    output = net.forward()
    
    # Post-process the output
    output = np.squeeze(output).transpose((1, 2, 0))
    output = np.clip(output, 0, 255).astype('uint8')
    
    return Image.fromarray(output)

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = '/dj/FSRCNN_x4.pb'  # Ensure you have the model file here
enhanced_image = enhance_image_with_fsrcnn(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


[ERROR:0@1164.452] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=1
[ERROR:0@1164.452] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 591 443 ]
[ERROR:0@1164.452] global net_impl.cpp:1176 getLayerShapesRecursively     blobs[0] = CV_32FC1 [ 56 1 5 5 ]
[ERROR:0@1164.452] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1721302265921/work/modules/dnn/src/layers/convolution_layer.cpp:397: error: (-215:Assertion failed) ngroups > 0 && inpCn % ngroups == 0 && outCn % ngroups == 0 in function 'getMemoryShapes'



error: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1721302265921/work/modules/dnn/src/layers/convolution_layer.cpp:397: error: (-215:Assertion failed) ngroups > 0 && inpCn % ngroups == 0 && outCn % ngroups == 0 in function 'getMemoryShapes'


In [6]:
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg16
from PIL import Image
import cv2
import numpy as np

# Function to enhance image using VGG16 pre-trained model for super resolution
def enhance_image_with_vgg16(image_path):
    # Load pre-trained VGG16 model
    model = vgg16(pretrained=True)
    model.eval()

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(image).unsqueeze(0)

    # Perform super-resolution
    with torch.no_grad():
        output_tensor = model.features(input_tensor)
        output_tensor = torch.nn.functional.interpolate(output_tensor, scale_factor=4, mode='bilinear', align_corners=False)
        output_tensor = output_tensor.clamp(0, 1)

    # Post-process the output
    output_image = transforms.ToPILImage()(output_tensor.squeeze())
    
    return output_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
enhanced_image = enhance_image_with_vgg16(image_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


/opt/conda/envs/ocr_env2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/ocr_env2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /data/ephemeral/home/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 68.7MB/s] 


ValueError: pic should not have > 4 channels. Got 512 channels.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image
import cv2

# Define the SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

# Function to enhance image using SRCNN
def enhance_image_with_srcnn(image_path, model_path):
    # Load pre-trained SRCNN model
    model = SRCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = ToTensor()(image).unsqueeze(0)

    # Perform super-resolution
    with torch.no_grad():
        output = model(image)

    # Post-process the output
    output_image = ToPILImage()(output.squeeze())

    return output_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = 'path/to/your/srcnn_model.pth'  # Ensure you have the model file here
enhanced_image = enhance_image_with_srcnn(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


/tmp/ipykernel_181103/633738435.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/srcnn_model.pth'

In [2]:
import cv2
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

def enhance_image_with_realesrgan(image_path):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError(f"Unable to load image at {image_path}")

    # Initialize the Real-ESRGAN model
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    model_path = '/dj/Real-ESRGAN/weights/RealESRGAN_x4.pth'
    realesrgan = RealESRGANer(scale=4, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=True)

    # Enhance image
    enhanced_image, _ = realesrgan.enhance(image)

    return enhanced_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
enhanced_image = enhance_image_with_realesrgan(image_path)
cv2.imwrite('/dj/enhanced_image.png', enhanced_image)
print("Enhanced image saved to '/dj/enhanced_image.png'")


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torchvision.models.detection import ssd300_vgg16
from PIL import Image
import cv2

# Function to enhance the image using a pre-trained super-resolution model
def enhance_image_with_super_resolution(image_path):
    # Load the pre-trained model from torchvision
    model = torch.hub.load('pytorch/vision:v0.10.0', 'srresnet', pretrained=True)
    model.eval()
    
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = preprocess(image).unsqueeze(0)
    
    # Enhance the image
    with torch.no_grad():
        output_tensor = model(input_tensor)
    
    # Post-process the output image
    output_tensor = output_tensor.squeeze(0).clamp(0, 1)
    output_image = transforms.ToPILImage()(output_tensor)
    
    return output_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
enhanced_image = enhance_image_with_super_resolution(image_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


Using cache found in /data/ephemeral/home/.cache/torch/hub/pytorch_vision_v0.10.0


RuntimeError: Cannot find callable srresnet in hubconf

In [2]:
import torch
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from PIL import Image

def enhance_image_with_realesrgan(image_path, model_path):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Initialize model
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)
    upsampler = RealESRGANer(
        scale=4,
        model_path=model_path,
        model=model,
        tile=0,
        tile_pad=10,
        pre_pad=0,
        half=False,
    )

    # Enhance image
    enhanced_image, _ = upsampler.enhance(image, outscale=4)

    # Convert the enhanced image to PIL format for saving
    enhanced_image = Image.fromarray(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))

    return enhanced_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = 'RealESRGAN_x4.pth'  # Ensure you have the model file here
enhanced_image = enhance_image_with_realesrgan(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

In [1]:
import cv2
import numpy as np
from PIL import Image

def enhance_image_with_espcn(image_path, model_path):
    # Load the pre-trained ESPCN model
    net = cv2.dnn.readNetFromTensorflow(model_path)
    
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(image.shape[1], image.shape[0]), mean=(0, 0, 0), swapRB=False, crop=False)
    
    # Perform super-resolution
    net.setInput(blob)
    output = net.forward()
    
    # Post-process the output
    output = np.squeeze(output).transpose((1, 2, 0))
    output = np.clip(output * 255.0, 0, 255).astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    
    # Convert to PIL Image
    enhanced_image = Image.fromarray(output)
    
    return enhanced_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
model_path = '/dj/FSRCNN_x4.pb'  # Ensure you have the model file here
enhanced_image = enhance_image_with_espcn(image_path, model_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


[ERROR:0@0.084] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=1
[ERROR:0@0.084] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 591 443 ]
[ERROR:0@0.084] global net_impl.cpp:1176 getLayerShapesRecursively     blobs[0] = CV_32FC1 [ 56 1 5 5 ]
[ERROR:0@0.084] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1721302265921/work/modules/dnn/src/layers/convolution_layer.cpp:397: error: (-215:Assertion failed) ngroups > 0 && inpCn % ngroups == 0 && outCn % ngroups == 0 in function 'getMemoryShapes'



error: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1721302265921/work/modules/dnn/src/layers/convolution_layer.cpp:397: error: (-215:Assertion failed) ngroups > 0 && inpCn % ngroups == 0 && outCn % ngroups == 0 in function 'getMemoryShapes'


In [1]:
import torch
import cv2
from torchvision.transforms import ToTensor, ToPILImage

def enhance_image_with_esrgan(image_path):
    # Load the pre-trained ESRGAN model from PyTorch Hub
    model = torch.hub.load('pytorch/vision:v0.10.0', 'esrgan', pretrained=True)
    model.eval()

    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = ToTensor()(image).unsqueeze(0)

    # Perform super-resolution
    with torch.no_grad():
        output = model(image)

    # Post-process the output
    output_image = ToPILImage()(output.squeeze())

    return output_image

# Example usage
image_path = 'dj/data/test/0a4f2decf34d3bff.jpg'
enhanced_image = enhance_image_with_esrgan(image_path)
enhanced_image.save('/dj/enhanced_image.png')
print("Enhanced image saved to '/dj/enhanced_image.png'")


Using cache found in /data/ephemeral/home/.cache/torch/hub/pytorch_vision_v0.10.0


RuntimeError: Cannot find callable esrgan in hubconf